<a href="https://colab.research.google.com/github/dajopr/lectures/blob/main/image_processing/lecture_07_camera_calibration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise: Camera Calibration and Undistortion

**Objective:** To understand and apply the camera calibration process using OpenCV to find camera intrinsic parameters and distortion coefficients. This exercise will cover preparing calibration patterns, finding corners, performing calibration, and applying the results to undistort an image.

**Background:**
Camera calibration is a crucial step in many computer vision applications. It involves determining the camera's intrinsic parameters (like focal length and optical centers) and distortion coefficients. OpenCV provides tools to perform calibration using images of a known pattern, such as a chessboard. Once calibrated, images can be undistorted to remove lens artifacts.

**0. Setup and Prerequisites:**
    * Ensure you have Python, OpenCV (`cv2`), NumPy (`numpy`), and Matplotlib (`matplotlib`) installed.
    * You will need a set of images of a chessboard pattern taken from various angles and distances. The tutorial suggests at least 10 good images. Ensure the entire chessboard is visible in each. The size of the chessboard (e.g., 9x6 squares) needs to be known.
    * Create a directory (e.g., `calibration_images/`) in the same location as this notebook and place your chessboard images there.

In [ ]:
import cv2
import numpy as np
import glob
import matplotlib.pyplot as plt

**1. Prepare Object Points and Image Points:**
    * **Object Points:**
        * Define the real-world coordinates of the 3D points on your chessboard. For a chessboard pattern, these points are the corners of the squares.
        * For a chessboard with `nx` by `ny` internal corners (e.g., for a 9x6 board, `nx=8, ny=5` if counting squares, or `nx=9, ny=6` if counting internal corners along edges), the coordinates can be set as `(0,0,0), (1,0,0), ..., (nx-1,0,0), (0,1,0), ..., (nx-1,ny-1,0)`. Assume the Z-coordinate is always 0 as the pattern is planar.
    * **Image Points:**
        * These will store the 2D coordinates of the detected corners in your calibration images.
    * Define the chessboard size (number of internal corners).

In [ ]:
# Define the dimensions of the chessboard (number of internal corners)
CHECKERBOARD_WIDTH = None  # Number of internal corners along the width
CHECKERBOARD_HEIGHT = None  # Number of internal corners along the height
CHECKERBOARD = (CHECKERBOARD_WIDTH, CHECKERBOARD_HEIGHT)

# Termination criteria for cornerSubPix
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# Prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(CHECKERBOARD_WIDTH-1, CHECKERBOARD_HEIGHT-1,0)
objp = np.zeros((CHECKERBOARD_HEIGHT * CHECKERBOARD_WIDTH, 3), np.float32)
objp[:, :2] = np.mgrid[0:CHECKERBOARD_WIDTH, 0:CHECKERBOARD_HEIGHT].T.reshape(-1, 2)
# You can multiply by square size here if you want units in mm or inches, e.g., objp = objp * square_size

# Arrays to store object points and image points from all the images.
objpoints = []  # 3d point in real world space
imgpoints = []  # 2d points in image plane.

# Path to your calibration images
# MAKE SURE YOU HAVE A FOLDER NAMED 'calibration_images' WITH YOUR IMAGES
images_path = (
    "images/chessboards/*.jpg"  # Adjust if your images are .png or other format
)
images = glob.glob(images_path)

if not images:
    print(f"No images found at {images_path}. Please check the path and image format.")
else:
    print(f"Found {len(images)} images for calibration.")

**2. Find Chessboard Corners:**
    * Load each calibration image.
    * Convert to grayscale.
    * Use `cv2.findChessboardCorners()`.
    * If corners are found, refine them using `cv2.cornerSubPix()` and store object/image points.
    * (Optional but recommended) Draw and display corners for verification.

**3. Perform Camera Calibration:**
    * Use `cv2.calibrateCamera()` with your `objpoints`, `imgpoints`, and the grayscale image shape.
    * Print the obtained camera matrix (`mtx`) and distortion coefficients (`dist`).

**4. Undistort an Image:**
    * Load one of your calibration images (or a new image).
    * Use `cv2.getOptimalNewCameraMatrix()` to refine the camera matrix.
    * Undistort the image using `cv2.undistort()`.
    * Display the original and undistorted images.

**5. Re-projection Error (Optional but Recommended):**
    * Calculate the re-projection error to evaluate the accuracy of your calibration.

**Questions & Discussion:**

1.  Why is it important to use multiple images of the chessboard from different viewpoints for calibration?
2.  What do the camera matrix (`mtx`) and distortion coefficients (`dist`) represent? Explain the main components of the camera matrix (fx, fy, cx, cy).
3.  What is the effect of the `alpha` parameter in `cv2.getOptimalNewCameraMatrix()`? Try running your undistortion with different `alpha` values (e.g., 0 and 1) and describe the differences in the output.
4.  If you calculated the re-projection error, what was your mean error? Is it generally considered acceptable for this type of calibration?
5.  The tutorial mentions two ways of undistortion: `cv2.undistort()` and using `cv2.initUndistortRectifyMap()` followed by `cv2.remap()`. What is the primary advantage of using the remapping method, especially for video processing?